<a href="https://colab.research.google.com/github/taahuu/mini-project-/blob/main/miniproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import socket
import struct

class Firewall:
    def __init__(self):
        self.rules = []

    def add_rule(self, action, rule_type, value):
      # Add a rule to the firewall. Rules specify what to do (action) for a given condition.
        self.rules.append((action, rule_type, value))

    def packet_matches_rule(self, pkt):
        for action, rule_type, value in self.rules:
            if rule_type == "protocol":
                if pkt.protocolname == value:
                    return action
            elif rule_type == "ipaddress":
                if pkt.srcipaddress == value:
                    return action
            elif rule_type == "portnum":
                if pkt.srcportnum == int(value):
                    return action
        return "pass"

    def handle_packet(self, pckt):
      # Extract packet information like source IP, source port, and protocol.
        ip_header = pckt[0:20]
        iph = struct.unpack('!BBHHHBBH4s4s', ip_header)

        version_ihl = iph[0]
        ihl = (version_ihl & 0xF) * 4
        protocol = iph[6]
        src_ip = socket.inet_ntoa(iph[8])
        dst_ip = socket.inet_ntoa(iph[9])

        if protocol == 6:
            src_port = struct.unpack('!H', pckt[ihl:ihl + 2])[0]
            return "tcp", src_ip, src_port
        elif protocol == 17:
            src_port = struct.unpack('!H', pckt[ihl:ihl + 2])[0]
            return "udp", src_ip, src_port
        elif protocol == 1:
            return "icmp", src_ip, None
        else:
            return "unknown", src_ip, None

def main():
    global firewall

    firewall = Firewall()

    # Add your firewall rules here
    firewall.add_rule("block", "protocol", "tcp")
    firewall.add_rule("block", "ipaddress", "192.168.1.100")
    firewall.add_rule("block", "portnum", "80")

    s = socket.socket(socket.AF_INET, socket.SOCK_RAW, socket.IPPROTO_TCP)

    while True:
        packet, addr = s.recvfrom(65565)
        protocol, src_ip, src_port = firewall.handle_packet(packet)
        action = firewall.packet_matches_rule(firewall)

        if action == "block":
            print(f"Blocked: {protocol} from {src_ip}:{src_port}")
        else:
            print(f"Accepted: {protocol} from {src_ip}:{src_port}")


In [4]:
#Detect the Xss attack
import re

# Define a regular expression pattern to detect potential XSS attacks
xss_pattern = re.compile(r'<\s*script[^>]*>[^<]*<\s*/\s*script\s*>', re.I)

def detect_xss_attack(request):
    # Check for the XSS pattern in the request URL, headers, and body
    if (
        re.search(xss_pattern, request.url) or
        any(re.search(xss_pattern, header) for header in request.headers.values()) or
        re.search(xss_pattern, request.body)
    ):
        return True
    return False

# Example usage:
class HttpRequest:
    def __init__(self, url, headers, body):
        self.url = url
        self.headers = headers
        self.body = body

request_url = "http://example.com/path?name=<script>alert('XSS')</script>"
request_headers = {"User-Agent": "Mozilla/5.0", "Referer": "<script>evil.com</script>"}
request_body = "payload=<script>alert('XSS')</script>"

http_request = HttpRequest(request_url, request_headers, request_body)
if detect_xss_attack(http_request):
    print("XSS attack detected")
    print("Blocked")
else:
    print("No XSS attack detected")


XSS attack detected
Blocked


In [5]:
import requests
from bs4 import BeautifulSoup

def get_all_forms(url):
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    return soup.find_all("form")

def is_vulnerable(response):
    errors = ["you have an error in your sql syntax;", "warning: mysql", "unclosed quotation mark after the character string"]
    for error in errors:
        if error in response.content.decode().lower():
            return True
    return False

def scan_sql_injection(url):
    for c in "\"'":
        new_url = f"{url}{c}"
        response = requests.get(new_url)
        if is_vulnerable(response):
            print(f"[+] SQL Injection vulnerability detected, link: {new_url}")
            return

if __name__ == "__main__":
    import sys
    if len(sys.argv) != 2:
        print("Usage: python sql_injection_detector.py <url>")
    else:
        url = sys.argv[1]
        scan_sql_injection(url)


Usage: python sql_injection_detector.py <url>
